In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import pandas as pd
from dotenv import load_dotenv
import io

In [2]:
load_dotenv()

True

In [29]:
client = genai.Client()

In [43]:
filepath = pathlib.Path("Z:\\Maitra & Co\\Clients\\S\\Sinha Niranjan & Sulochana\\Sinha Sulochana\\AY 25-26\\Income tax\\Bank Statement\\automation_image_(don't_touch).pdf")

In [44]:
myfile = client.files.upload(file=filepath)

In [33]:
prompt = """
Extract all transactions from the provided financial document (statement or passbook) into a raw CSV string.

**Output Schema & Headers (7 columns, exact order):**
`Date,ChequeNo,Narration,ValueDate,WithdrawalAmount,DepositAmount,ClosingBalance`

**Processing Rules:**
- **Combine Multi-line Narration:** Merge multi-line transaction descriptions (like 'Particulars' or 'Narration') into a single field with spaces.
- **Handle Missing Columns:** If a source document has no 'ValueDate', keep the column in the header but leave its data fields empty.
- **Data Cleaning:** Remove all non-numeric characters from amount columns (e.g., '₹', ',', 'Cr', 'Dr'). `1,234.56 Cr` must become `1234.56`.
- **Zero Values:** Represent empty or zero withdrawals/deposits as `0.00`.
- **CRITICAL COMMA RULE:** If any field's text contains a comma, enclose that entire field in double quotes. Example: `...,"Transfer, Savings Account",...`
- **Exclusions:** Do not include any summary or footer lines (e.g., 'Clear Balance', 'Carried Forward', 'We provide ATM Cards...').

**Final Output:**
- Raw CSV text only.
- No explanations, summaries, or markdown ` ``` `.
- Start directly with the header row.
"""

In [45]:
response = client.models.generate_content(
  model="gemini-2.5-flash-preview-04-17",
  contents=[prompt, myfile])

In [46]:
print(response.text)

Date,ChequeNo,Narration,ValueDate,WithdrawalAmount,DepositAmount,ClosingBalance
19/04/24,,INT ON QIDR3620367554,,0.00,13050.00,111140.15
29/04/24,,INT ON QIDR3534651750,,0.00,3263.00,114403.15
04/05/24,,INT ON QIDR3476848163,,0.00,8157.00,122560.15
25/05/24,,INT ON SB,,0.00,740.00,123300.15
31/05/24,,INT ON QIDR3562491145,,0.00,4639.00,127939.15
01/07/24,,WDL TFR Via Standing I,,2714.00,0.00,125225.15
12/07/24,,INT ON QIDR3657106784,,0.00,157.00,125382.15
19/07/24,,INT ON QIDR3620367554,,0.00,13050.00,138432.15
29/07/24,,INT ON QIDR3534651750,,0.00,3263.00,141695.15
04/08/24,,INT ON QIDR347684816,,0.00,8156.00,149851.15
24/08/24,,INT ON SR,,0.00,937.00,150788.15
31/08/24,,INT ON QIDR3562491145,,0.00,4640.00,155428.15
12/10/24,,INT ON QIDR3657106781,,0.00,158.00,155586.15
19/10/24,,INT ON QIDR3620367554,,0.00,13230.00,168816.15
29/10/24,,INT ON QIDR3534651750,,0.00,3262.00,172078.15
04/11/24,,INT. ON QIDR3476848163,,0.00,8270.00,180348.15
23/11/24,,INT ON SB3021,,0.00,1139.00,181487.15


In [47]:
client.files.delete(name=myfile.name)

DeleteFileResponse()

In [48]:
doc = io.StringIO(response.text)

In [49]:
df_psb = pd.read_csv(doc, sep=',',engine='python')
print(df_psb.tail(5))

        Date  ChequeNo              Narration  ValueDate  WithdrawalAmount  \
19  12/01/25       NaN  INT ON QIDR3657106781        NaN               0.0   
20  19/01/25       NaN  INT ON QIDR3620367554        NaN               0.0   
21  29/01/25       NaN  INT ON QIDR3534651750        NaN               0.0   
22  04/02/25       NaN  INT ON QIDR3476848163        NaN               0.0   
23  22/02/25       NaN              INT ON SB        NaN               0.0   

    DepositAmount  ClosingBalance  
19          157.0       986285.15  
20        13230.0       999515.15  
21         3263.0      1002778.15  
22         8269.0      1011047.15  
23         5586.0      1016633.15  
